First we set up the notebook and import our tools.  I will run this book on kernal PUI2016_Python3

In [2]:
from __future__ import print_function, division
import geopandas as gp
import pandas as pd
import pylab as pl
import os
import json
import seaborn as sns
import matplotlib as plt
import numpy as np
import sys

#s = json.load( open(os.getenv('PUI2016')+'/fbb_matplotlibrc.json') )
#pl.rcParams.update(s)
if os.getenv("PUIDATA") is None:
    print ("$PUIDATA to point to set PUIdata dir")

%pylab inline
print(os.getenv("PUIDATA"))

Populating the interactive namespace from numpy and matplotlib
/home/cusp/cjs676/PUIdata


/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Below is the function which will get and move the citibike data

In [3]:


#check if the env variable PUIDATA exists
if os.getenv("PUIDATA") is None:
    print ("must set PUIDATA env variable")
    sys.exit()

def getCitiBikeCSV(datestring):
    '''Downloads citibike data and unzips it. If the data is downloaded by not unzippeds it zips it. Moves the data to $PUIDATA
    Arguments:
        date string as yyyymm
    '''
    print ("Downloading", datestring)
    ### First I will heck that it is not already there
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        if os.path.isfile(datestring + "-citibike-tripdata.csv"):
            # if in the current dir just move it
            if os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA")):
                print ("Error moving file!, Please check!")
        #otherwise start looking for the zip file
        else:
            if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip"):
                if not os.path.isfile(datestring + "-citibike-tripdata.zip"):
                    os.system("curl -O https://s3.amazonaws.com/tripdata/" + datestring + "-citibike-tripdata.zip")
                ###  To move it I use the os.system() functions to run bash commands with arguments
                os.system("mv " + datestring + "-citibike-tripdata.zip " + os.getenv("PUIDATA"))
            ### unzip the csv 
            os.system("unzip " + os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip")
            ## NOTE: old csv citibike data had a different name structure. 
            if '2014' in datestring:
                os.system("mv " + datestring[:4] + '-' +  datestring[4:] + 
                          "\ -\ Citi\ Bike\ trip\ data.csv " + datestring + "-citibike-tripdata.csv")
            os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA"))
    ### One final check:
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        print ("WARNING!!! something is wrong: the file is not there!")

    else:
        print ("file in place, you can continue")


Now we will get the data 201506 first #edit:  success, checked with the Terminal

In [4]:
datestring = '201506'
getCitiBikeCSV(datestring)

file in place, you can continue


###### Now we will get the 201601 data #edit success, checked with the Terminal


In [5]:
datestring = '201601'
getCitiBikeCSV(datestring)

file in place, you can continue


Now we will load the citibike data into datframes
cb15 = 201506 data
cb16 = 201601 data

In [7]:
cb15 = pd.read_csv((os.getenv("PUIDATA") + "/" + '201506' + "-citibike-tripdata.csv"))

#The line line was to test if the path was proper. It is now commented out
#print((os.getenv("PUIDATA") + "/" + '201506' + "-citibike-tripdata.csv"))

In [8]:
#Check to see if the file was read
cb15.head(10)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1338,6/1/2015 0:00,6/1/2015 0:22,128,MacDougal St & Prince St,40.727103,-74.002971,2021,W 45 St & 8 Ave,40.759291,-73.988597,20721,Subscriber,1984.0,1
1,290,6/1/2015 0:00,6/1/2015 0:05,438,St Marks Pl & 1 Ave,40.727791,-73.985649,312,Allen St & E Houston St,40.722055,-73.989111,21606,Subscriber,1997.0,1
2,634,6/1/2015 0:01,6/1/2015 0:11,383,Greenwich Ave & Charles St,40.735238,-74.000271,388,W 26 St & 10 Ave,40.749718,-74.002950,16595,Subscriber,1993.0,1
3,159,6/1/2015 0:01,6/1/2015 0:04,361,Allen St & Hester St,40.716059,-73.991908,531,Forsyth St & Broome St,40.718939,-73.992663,16949,Subscriber,1981.0,1
4,1233,6/1/2015 0:02,6/1/2015 0:22,382,University Pl & E 14 St,40.734927,-73.992005,532,S 5 Pl & S 4 St,40.710451,-73.960876,17028,Customer,NaN,0
5,690,6/1/2015 0:03,6/1/2015 0:15,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,410,Suffolk St & Stanton St,40.720664,-73.985180,20857,Subscriber,1976.0,1
6,766,6/1/2015 0:04,6/1/2015 0:17,346,Bank St & Hudson St,40.736529,-74.006180,2003,1 Ave & E 18 St,40.733812,-73.980544,15978,Subscriber,1976.0,1
7,1160,6/1/2015 0:06,6/1/2015 0:25,512,W 29 St & 9 Ave,40.750073,-73.998393,445,E 10 St & Avenue A,40.727408,-73.981420,17306,Subscriber,1962.0,2
8,550,6/1/2015 0:07,6/1/2015 0:16,525,W 34 St & 11 Ave,40.755942,-74.002116,447,8 Ave & W 52 St,40.763707,-73.985162,18392,Subscriber,1973.0,1
9,545,6/1/2015 0:08,6/1/2015 0:17,403,E 2 St & 2 Ave,40.725029,-73.990697,439,E 4 St & 2 Ave,40.726281,-73.989780,16300,Subscriber,1958.0,1


Repeating the process for the next file.

In [10]:
cb16 = pd.read_csv((os.getenv("PUIDATA") + "/" + '201601' + "-citibike-tripdata.csv"))

#The line line was to test if the path was proper. It is now commented out
#print((os.getenv("PUIDATA") + "/" + '201601' + "-citibike-tripdata.csv"))

In [11]:
cb16.head(5)


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,923,1/1/2016 00:00:41,1/1/2016 00:16:04,268,Howard St & Centre St,40.719105,-73.999733,3002,South End Ave & Liberty St,40.711512,-74.015756,22285,Subscriber,1958.0,1
1,379,1/1/2016 00:00:45,1/1/2016 00:07:04,476,E 31 St & 3 Ave,40.743943,-73.979661,498,Broadway & W 32 St,40.748549,-73.988084,17827,Subscriber,1969.0,1
2,589,1/1/2016 00:00:48,1/1/2016 00:10:37,489,10 Ave & W 28 St,40.750664,-74.001768,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,21997,Subscriber,1982.0,2
3,889,1/1/2016 00:01:06,1/1/2016 00:15:56,268,Howard St & Centre St,40.719105,-73.999733,3002,South End Ave & Liberty St,40.711512,-74.015756,22794,Subscriber,1961.0,2
4,1480,1/1/2016 00:01:12,1/1/2016 00:25:52,2006,Central Park S & 6 Ave,40.765909,-73.976342,2006,Central Park S & 6 Ave,40.765909,-73.976342,14562,Subscriber,1952.0,1


#Success.  
Now to try to get the IRS data. 
From investigation on the IRS page: File 14zp33ny.xls = https://www.irs.gov/pub/irs-soi/14zp33ny.xls


In [12]:
#First attempt.  Trying to use the hint =on the midterm

#datestring = '201601'
#getCitiBikeCSV(datestring)

def readIncomeXls(url):
    return pd.read_excel(url, header=3, index_col="ZIP\ncode [1]")

In [14]:
#Lets see what our function returns
url = 'https://www.irs.gov/pub/irs-soi/14zp33ny.xls'
readIncomeXls(url)

,Size of adjusted gross income,Number of returns,Number of single returns,Number of joint returns,Number of head of household returns,Number with paid preparer's signature,Number of exemptions,Number of dependents,Number of volunteer prepared returns [2],Unnamed: 10,...,Total tax liability [9],Unnamed: 116,Additional Medicare tax,Unnamed: 118,Net investment income tax,Unnamed: 120,Tax due at time of filing [10],Unnamed: 122,Overpayments refunded [11],Unnamed: 124
ZIP code [1],,,,,,,,,,,,,,,,,,,,,
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Total,Number of volunteer income tax assistance (VIT...,...,Number of returns,Amount,Number of returns,Amount,Number of returns,Amount,Number of returns,Amount,Number of returns,Amount
NaN,NaN,-1.0,-2.0,-3.0,-4.0,-5.0,-6.0,-7.0,-8,-9,...,-114,-115,-116,-117,-118,-119,-120,-121,-122,-123
0,Total,9397410.0,4750550.0,2942890.0,1515950.0,6110600.0,17322490.0,5539120.0,279510,197150,...,7229610,129761061,280780,900175,300410,2745701,1781540,10177242,7115810,25052463
0,"$1 under $25,000",3617400.0,2432240.0,468500.0,665410.0,2247810.0,5220510.0,1678560.0,187260,136800,...,1804270,1677200,0,0,0,0,451080,361017,2886740,6049923
0,"$25,000 under $50,000",2070870.0,1103290.0,449390.0,467540.0,1280840.0,3729240.0,1219770.0,69390,49620,...,1769800,4790944,0,0,0,0,287080,461908,1747120,4510472
0,"$50,000 under $75,000",1248170.0,590760.0,417090.0,203970.0,811480.0,2354290.0,690410.0,15750,8270,...,1208460,7172107,30,10,0,0,258590,569921,961470,2794612
0,"$75,000 under $100,000",805170.0,280040.0,408350.0,94660.0,547720.0,1722440.0,509330.0,4250,1340,...,796620,7548905,60,8,0,0,188570,539823,592800,2149986
0,"$100,000 under $200,000",1171760.0,261640.0,818600.0,70860.0,834560.0,2953670.0,963610.0,2600,970,...,1166800,21520383,13620,3189,4900,1158,352230,1681115,766840,3660123
0,"$200,000 or more",484040.0,82580.0,380960.0,13510.0,388190.0,1342340.0,477440.0,260,150,...,483660,87051522,267070,896968,295510,2744543,243990,6563458,160840,5887347


In [ ]:
14zp33ny.xls